In [1]:
import pandas_datareader.data as web #to collect data
import datetime as dt #to specify start and end dates


# import yfinance as yf

import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

from patsy import dmatrices
from tqdm.notebook import tqdm
tqdm.pandas()

pd.set_option('display.max_columns', 500)

## Data Reading

### Company Keys

In [2]:
companyKeys = pd.read_excel("Prowess IQ - Indicators.xlsx").rename({"Company Name":"CompanyName", "Prowess company code":"ProwessCode", "NSE symbol":"Symbol"}, axis = 1 )

In [3]:
companyKeys

,CompanyName,ProwessCode,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,Symbol,Head office address
0,'K' Steamship Agencies Pvt. Ltd.,3,Private Ltd.,10203010000,1971.0,Diversified,1.100000e+14,Diversified,34.0,20102000000,Private (Indian),Between 1951 and 1971,NaN,NaN
1,'X'Clusive Business Centre Pvt. Ltd.,307865,Private Ltd.,10203010000,1996.0,Diversified financial services,1.022000e+14,"Other financial service activities, except ins...",649.0,20102000000,Private (Indian),Between 1991 and 2013,NaN,NaN
2,1 To 1 Help.Net Pvt. Ltd.,591675,Private Ltd.,10203010000,2001.0,Business services & consultancy,1.010415e+14,Management consultancy activities,70200.0,20102000000,Private (Indian),Between 1991 and 2013,NaN,NaN
3,10C India Internet Pvt. Ltd.,556976,Private Ltd.,10203010000,2012.0,Telecommunication services,1.010406e+14,Other telecommunications activities,61900.0,20102000000,Private (Indian),Between 1991 and 2013,NaN,NaN
4,10I Commerce Services Pvt. Ltd.,560502,Private Ltd.,10203010000,2015.0,Wholesale trading,1.010404e+14,"Wholesale of radio, television and other consu...",46522.0,20102000000,Private (Indian),After 2014,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55316,Zylog Systems Ltd.,275793,Public Ltd.,10203020000,1995.0,Computer software,1.010408e+14,Providing software support and maintenance to ...,62013.0,20102000000,Private (Indian),Between 1991 and 2013,ZYLOG,NaN
55317,Zyma Laboratories Ltd. [Merged],275794,Public Ltd.,10203020000,1970.0,Other fund based financial services,1.020400e+14,"Trusts, funds and other financial vehicles",64300.0,20102000000,Private (Indian),Between 1951 and 1971,NaN,NaN
55318,Zyphar'S Pharmaceutics Pvt. Ltd.,565342,Private Ltd.,10203010000,2006.0,Wholesale trading,1.010404e+14,Wholesale of pharmaceutical and medical goods,46497.0,20102000000,Private (Indian),Between 1991 and 2013,NaN,NaN
55319,Zytel Agencies Ltd.,275795,Public Ltd.,10203020000,1984.0,Other fund based financial services,1.020400e+14,"Trusts, funds and other financial vehicles",64300.0,20102000000,Private (Indian),Between 1972 and 1985,NaN,NaN


### Director Level Data

In [4]:
dirLevel = pd.read_csv("Main_Director_complete v140924.csv").drop(labels = ["Unnamed: 0"], axis = 1)

C:\Users\SHIVAM\AppData\Local\Temp\ipykernel_20924\676584468.py:1: DtypeWarning: Columns (28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dirLevel = pd.read_csv("Main_Director_complete v140924.csv").drop(labels = ["Unnamed: 0"], axis = 1)


In [5]:
dirLevel["Appointment Date"] = pd.to_datetime(dirLevel["Appointment Date"], format = "%Y-%m-%d")
dirLevel

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal,IsFirstTermIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,AsOnYear,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaN,NaN,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"(('3M INDIA LTD.', 'U'),)","(('3M INDIA LTD.', 'U'),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,64.0,0.0,3.687671,0,0,0,0,2006,0,0,0,0
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaN,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0,(),"(('ASHOK LEYLAND LTD.', 'U'), ('3M INDIA LTD.'...","(('ASHOK LEYLAND LTD.', 'U'), ('3M INDIA LTD.'...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,84.0,0.0,4.263014,0,1,0,0,2006,0,0,0,0
2,ASHOKLEY,ASHOK LEYLAND LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2000-07-27,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaN,"MR D J BALAJI RAO, IS A FORMER DEPUTY MANAGING...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,2073.0,0,0,2073.0,2438.0,0,0,2438.0,2073.0,3289.0,(),"(('ASHO

In [6]:
dirLevel2 = dirLevel.merge(companyKeys, on = ["Symbol"], how = "left")

In [7]:
dirLevel2

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal,IsFirstTermIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,AsOnYear,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,ProwessCode,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,Head office address
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaN,NaN,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"(('3M INDIA LTD.', 'U'),)","(('3M INDIA LTD.', 'U'),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,64.0,0.0,3.687671,0,0,0,0,2006,0,0,0,0,3M India Ltd.,36277.0,Public Ltd.,1.020302e+10,1987.0,"Plastic furniture, floorings & miscellaneous i...",1.010120e+14,Manufacture of other plastics products n.e.c.,22209.0,2.020200e+10,Private (Foreign),Between 1986 and 1990,NaN
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaN,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0,(),"(('ASHOK LEYLAND LTD.', 'U'), ('3M INDIA LTD.'...","(('ASHOK LEYLAND LTD.', 'U'), ('3M INDIA LTD.'...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,84.0,0.0,4.263014,0,1,0,0,2006,0,0,0,0,3M India Ltd.,36277.0,Public Ltd.,1.020302e+10,1987.0,"Plastic furniture, floorings & miscellaneous i...",1.010120e+14,Manufacture of other plastics products n.e.c.,22209.0,2.020200e+10,Private (Foreign),Between 1986 and 1990,NaN
2,ASHOKLEY,ASHOK 

## 120 CAR

In [8]:
output120CAR = pd.read_csv("output120CAR.csv")

In [9]:
output120CAR.columns

Index(['Unnamed: 0', 'CompanyName', 'AsOnDate', 'ACP', 'pct', 'RF', 'RMRF',
       'MF', 'SMB', 'HML', 'OLS120_intercept', 'OLS120_RMRF', 'OLS120_SMB',
       'OLS120_HML', 'OLS120_r_squared', 'OLS120_adjusted_r_squared',
       'OLS120_f_p_value', 'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum',
       'SMB_cen3sum', 'HML_cen3sum', 'pct_cen5sum', 'RF_cen5sum',
       'RMRF_cen5sum', 'SMB_cen5sum', 'HML_cen5sum', 'pct_cen7sum',
       'RF_cen7sum', 'RMRF_cen7sum', 'SMB_cen7sum', 'HML_cen7sum',
       'pct_cen11sum', 'RF_cen11sum', 'RMRF_cen11sum', 'SMB_cen11sum',
       'HML_cen11sum', '120CAR3', '120CAR5', '120CAR7', '120CAR11'],
      dtype='object')

In [14]:
colDrop = output120CAR.columns[[0, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                      28, 29, 30, 31, 32, 33, 34, 35, 36]]
colDrop

Index(['Unnamed: 0', 'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum',
       'SMB_cen3sum', 'HML_cen3sum', 'pct_cen5sum', 'RF_cen5sum',
       'RMRF_cen5sum', 'SMB_cen5sum', 'HML_cen5sum', 'pct_cen7sum',
       'RF_cen7sum', 'RMRF_cen7sum', 'SMB_cen7sum', 'HML_cen7sum',
       'pct_cen11sum', 'RF_cen11sum', 'RMRF_cen11sum', 'SMB_cen11sum',
       'HML_cen11sum'],
      dtype='object')

In [15]:
output120CAR2 = output120CAR.drop(labels = colDrop, axis = 1)
output120CAR2

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,120CAR3,120CAR5,120CAR7,120CAR11
0,20 Microns Ltd.,2009-04-27,10.85,-0.011840,0.027124,-0.703543,-0.676419,0.213648,0.770117,-0.024856,0.012905,0.011293,0.001436,0.274365,0.255435,4.519505e-08,0.240052,0.094556,0.133024,0.333801
1,20 Microns Ltd.,2009-04-28,12.57,0.158525,0.009040,-3.021266,-3.012226,-0.063645,0.498647,-0.024314,0.013002,0.011581,0.002395,0.275674,0.256942,3.488935e-08,0.136922,0.075586,-0.014620,0.334596
2,20 Microns Ltd.,2009-04-29,11.80,-0.061257,0.008932,2.488979,2.497912,-1.165069,0.474631,-0.023559,0.012881,0.011145,0.003962,0.282966,0.264422,1.964126e-08,0.045605,0.024116,0.055896,0.110934
3,20 Microns Ltd.,2009-05-04,11.32,-0.040678,0.044670,3.920962,3.965632,-1.539316,0.596610,-0.024365,0.011572,0.008292,0.005490,0.248402,0.228964,2.834241e-07,-0.170596,0.036668,0.029613,-0.061727
4,20 Microns Ltd.,2009-05-05,11.23,-0.007951,0.008932,0.841529,0.850462,0.529611,-0.088872,-0.023842,0.010647,0.007331,0.007090,0.218311,0.198095,2.600825e-06,-0.101277,-0.162550,0.041268,0.033901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12986007,Zylog Systems Ltd.,2024-03-20,0.35,0.000000,0.018213,0.187741,0.205954,-0.677517,0.296181,-0.026576,0.003888,0.003108,-0.002064,0.043184,0.018438,1.616497e-01,0.026571,0.004357,-0.033668,0.000899
12986008,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026702,0.003907,0.003905,-0.001484,0.049581,0.025001,1.153618e-01,0.016759,-0.011482,-0.029841,NaN
12986009,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026628,0.004073,0.003156,-0.001910,0.044392,0.019678,1.517656e-01,-0.034759,-0.017161,0.003050,NaN
12986010,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026633,0.004079,0.003177,-0.001971,0.045188,0.020494,1.455639e-01,-0.030633,-0.020203,NaN,NaN


In [16]:
output120CAR2["AsOnDate"] = pd.to_datetime(output120CAR2["AsOnDate"], format = "%Y-%m-%d")

In [17]:
dir120CAR = dirLevel2.merge(output120CAR2, how = "left", left_on = ["CompanyName", "Appointment Date"], right_on = ["CompanyName", "AsOnDate"]).drop("AsOnDate_y", axis = 1).rename({"AsOnDate_x":"AsOnDate"}, axis = 1)

In [18]:
dir120CAR

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal,IsFirstTermIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,AsOnYear,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,ProwessCode,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,Head office address,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,120CAR3,120CAR5,120CAR7,120CAR11
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaN,NaN,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"(('3M INDIA LTD.', 'U'),)","(('3M INDIA LTD.', 'U'),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,64.0,0.0,3.687671,0,0,0,0,2006,0,0,0,0,3M India Ltd.,36277.0,Public Ltd.,1.020302e+10,1987.0,"Plastic furniture, floorings & miscellaneous i...",1.010120e+14,Manufacture of other plastics products n.e.c.,22209.0,2.020200e+10,Private (Foreign),Between 1986 and 1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaN,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0,(),"(('ASHOK LEYLAND LTD.', 'U'), ('3M INDIA LTD.'...","(('ASHOK LEYLAND LTD.', 'U'), ('3M INDIA LTD.'...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,84.0,0.0,4.263014,0,1,0,0,2006,0,0,0,0,3M I

In [19]:
dir120CAR.to_csv("dir120CAR.csv")

## 150 CAR

In [8]:
output150CAR = pd.read_csv("output150CAR.csv")

In [9]:
output150CAR.columns

Index(['Unnamed: 0', 'CompanyName', 'AsOnDate', 'ACP', 'pct', 'RF', 'RMRF',
       'MF', 'SMB', 'HML', 'OLS150_intercept', 'OLS150_RMRF', 'OLS150_SMB',
       'OLS150_HML', 'OLS150_r_squared', 'OLS150_adjusted_r_squared',
       'OLS150_f_p_value', 'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum',
       'SMB_cen3sum', 'HML_cen3sum', 'pct_cen5sum', 'RF_cen5sum',
       'RMRF_cen5sum', 'SMB_cen5sum', 'HML_cen5sum', 'pct_cen7sum',
       'RF_cen7sum', 'RMRF_cen7sum', 'SMB_cen7sum', 'HML_cen7sum',
       'pct_cen11sum', 'RF_cen11sum', 'RMRF_cen11sum', 'SMB_cen11sum',
       'HML_cen11sum', '150CAR3', '150CAR5', '150CAR7', '150CAR11'],
      dtype='object')

In [10]:
colDrop = output150CAR.columns[[0, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                      28, 29, 30, 31, 32, 33, 34, 35, 36]]
colDrop

Index(['Unnamed: 0', 'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum',
       'SMB_cen3sum', 'HML_cen3sum', 'pct_cen5sum', 'RF_cen5sum',
       'RMRF_cen5sum', 'SMB_cen5sum', 'HML_cen5sum', 'pct_cen7sum',
       'RF_cen7sum', 'RMRF_cen7sum', 'SMB_cen7sum', 'HML_cen7sum',
       'pct_cen11sum', 'RF_cen11sum', 'RMRF_cen11sum', 'SMB_cen11sum',
       'HML_cen11sum'],
      dtype='object')

In [11]:
output150CAR2 = output150CAR.drop(labels = colDrop, axis = 1)
output150CAR2

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,150CAR3,150CAR5,150CAR7,150CAR11
0,20 Microns Ltd.,2009-06-10,13.10,-0.045885,0.009040,1.510817,1.519858,-1.451946,-1.227270,-0.022698,0.009631,0.011863,0.003928,0.177787,0.160776,0.000003,-0.046697,-0.055617,-0.052366,0.068593
1,20 Microns Ltd.,2009-06-11,13.15,0.003817,0.009040,-0.767515,-0.758474,-0.512197,0.409118,-0.022708,0.009625,0.011863,0.003875,0.179487,0.162627,0.000002,0.003193,-0.072104,-0.041220,0.026291
2,20 Microns Ltd.,2009-06-12,12.63,-0.039544,0.009040,-1.045692,-1.036651,-1.058056,-0.482198,-0.022312,0.009421,0.011311,0.004019,0.174185,0.157216,0.000004,0.001168,0.014190,-0.027405,0.016287
3,20 Microns Ltd.,2009-06-15,12.13,-0.039588,0.027124,-2.156840,-2.129716,0.051534,0.030317,-0.022029,0.008960,0.010482,0.004496,0.158152,0.140854,0.000014,0.009269,0.044672,0.013614,-0.018708
4,20 Microns Ltd.,2009-06-16,12.55,0.034625,0.009040,1.225517,1.234557,0.082530,-0.770160,-0.021241,0.008279,0.009843,0.005729,0.138642,0.120943,0.000070,0.016732,0.004062,0.059300,-0.024349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12878967,Zylog Systems Ltd.,2024-03-20,0.35,0.000000,0.018213,0.187741,0.205954,-0.677517,0.296181,-0.026710,0.002660,0.003778,-0.001954,0.036481,0.016683,0.142006,0.026912,0.005548,-0.032931,0.002437
12878968,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026475,0.002737,0.003711,-0.001810,0.036499,0.016701,0.141841,0.019359,-0.011318,-0.029693,NaN
12878969,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026415,0.002826,0.003197,-0.002014,0.032462,0.012581,0.184321,-0.032738,-0.014879,0.003861,NaN
12878970,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026460,0.002981,0.003214,-0.002013,0.033903,0.014052,0.167967,-0.030156,-0.017667,NaN,NaN


In [12]:
output150CAR2["AsOnDate"] = pd.to_datetime(output150CAR2["AsOnDate"], format = "%Y-%m-%d")

In [13]:
dir150CAR = dirLevel2.merge(output150CAR2, how = "left", left_on = ["CompanyName", "Appointment Date"], right_on = ["CompanyName", "AsOnDate"]).drop("AsOnDate_y", axis = 1).rename({"AsOnDate_x":"AsOnDate"}, axis = 1)

In [14]:
dir150CAR

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal,IsFirstTermIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,AsOnYear,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,ProwessCode,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,Head office address,ACP,pct,RF,RMRF,MF,SMB,HML,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,150CAR3,150CAR5,150CAR7,150CAR11
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaN,NaN,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"(('3M INDIA LTD.', 'U'),)","(('3M INDIA LTD.', 'U'),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,64.0,0.0,3.687671,0,0,0,0,2006,0,0,0,0,3M India Ltd.,36277.0,Public Ltd.,1.020302e+10,1987.0,"Plastic furniture, floorings & miscellaneous i...",1.010120e+14,Manufacture of other plastics products n.e.c.,22209.0,2.020200e+10,Private (Foreign),Between 1986 and 1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaN,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0,(),"(('ASHOK LEYLAND LTD.', 'U'), ('3M INDIA LTD.'...","(('ASHOK LEYLAND LTD.', 'U'), ('3M INDIA LTD.'...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,84.0,0.0,4.263014,0,1,0,0,2006,0,0,0,0,3M I

In [15]:
dir150CAR.to_csv("dir150CAR.csv")

## 180 CAR

In [8]:
output180CAR = pd.read_csv("output180CAR.csv")

In [9]:
output180CAR.columns

Index(['Unnamed: 0', 'CompanyName', 'AsOnDate', 'ACP', 'pct', 'RF', 'RMRF',
       'MF', 'SMB', 'HML', 'OLS180_intercept', 'OLS180_RMRF', 'OLS180_SMB',
       'OLS180_HML', 'OLS180_r_squared', 'OLS180_adjusted_r_squared',
       'OLS180_f_p_value', 'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum',
       'SMB_cen3sum', 'HML_cen3sum', 'pct_cen5sum', 'RF_cen5sum',
       'RMRF_cen5sum', 'SMB_cen5sum', 'HML_cen5sum', 'pct_cen7sum',
       'RF_cen7sum', 'RMRF_cen7sum', 'SMB_cen7sum', 'HML_cen7sum',
       'pct_cen11sum', 'RF_cen11sum', 'RMRF_cen11sum', 'SMB_cen11sum',
       'HML_cen11sum', '180CAR3', '180CAR5', '180CAR7', '180CAR11'],
      dtype='object')

In [10]:
colDrop = output180CAR.columns[[0, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                      28, 29, 30, 31, 32, 33, 34, 35, 36]]
colDrop

Index(['Unnamed: 0', 'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum',
       'SMB_cen3sum', 'HML_cen3sum', 'pct_cen5sum', 'RF_cen5sum',
       'RMRF_cen5sum', 'SMB_cen5sum', 'HML_cen5sum', 'pct_cen7sum',
       'RF_cen7sum', 'RMRF_cen7sum', 'SMB_cen7sum', 'HML_cen7sum',
       'pct_cen11sum', 'RF_cen11sum', 'RMRF_cen11sum', 'SMB_cen11sum',
       'HML_cen11sum'],
      dtype='object')

In [11]:
output180CAR2 = output180CAR.drop(labels = colDrop, axis = 1)
output180CAR2

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,180CAR3,180CAR5,180CAR7,180CAR11
0,20 Microns Ltd.,2009-07-22,11.60,-0.004292,0.008824,-0.838877,-0.830052,1.315390,-0.113483,-0.021264,0.009618,0.012246,0.004820,0.191707,0.177851,3.880499e-08,0.030511,0.023964,0.100263,0.039433
1,20 Microns Ltd.,2009-07-23,11.85,0.021552,0.008824,2.213316,2.222140,-0.356213,-0.135894,-0.021177,0.009572,0.012166,0.004828,0.191666,0.177887,3.513662e-08,0.030030,0.113504,0.059542,0.065898
2,20 Microns Ltd.,2009-07-24,12.10,0.021097,0.008824,1.357015,1.365840,0.309985,1.036279,-0.020713,0.009391,0.011592,0.005322,0.187514,0.173665,5.458318e-08,0.100021,0.065523,0.081083,0.104323
3,20 Microns Ltd.,2009-07-27,13.23,0.093388,0.026475,0.453473,0.479948,0.904784,0.281945,-0.020111,0.008978,0.010864,0.005588,0.171890,0.157774,2.800550e-07,0.047778,0.068341,0.067241,0.101629
4,20 Microns Ltd.,2009-07-28,12.82,-0.030990,0.008824,0.610200,0.619024,1.065814,-0.348656,-0.019722,0.008387,0.010366,0.007027,0.157005,0.142635,1.287276e-06,0.039994,0.047974,0.103321,0.160682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12772601,Zylog Systems Ltd.,2024-03-20,0.35,0.000000,0.018213,0.187741,0.205954,-0.677517,0.296181,-0.026544,0.002821,0.002980,-0.002220,0.028359,0.011797,1.662029e-01,0.026798,0.005370,-0.033155,0.000599
12772602,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026511,0.002663,0.002803,-0.002113,0.025498,0.008887,2.071184e-01,0.020729,-0.010770,-0.029542,NaN
12772603,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026262,0.002406,0.002773,-0.002086,0.024682,0.008058,2.203633e-01,-0.032129,-0.014710,0.003200,NaN
12772604,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026266,0.002386,0.002795,-0.002064,0.024686,0.008062,2.202977e-01,-0.030409,-0.016975,NaN,NaN


In [12]:
output180CAR2["AsOnDate"] = pd.to_datetime(output180CAR2["AsOnDate"], format = "%Y-%m-%d")

In [13]:
dir180CAR = dirLevel2.merge(output180CAR2, how = "left", left_on = ["CompanyName", "Appointment Date"], right_on = ["CompanyName", "AsOnDate"]).drop("AsOnDate_y", axis = 1).rename({"AsOnDate_x":"AsOnDate"}, axis = 1)

In [14]:
dir180CAR

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal,IsFirstTermIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,AsOnYear,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,ProwessCode,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,Head office address,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,180CAR3,180CAR5,180CAR7,180CAR11
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaN,NaN,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"(('3M INDIA LTD.', 'U'),)","(('3M INDIA LTD.', 'U'),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,64.0,0.0,3.687671,0,0,0,0,2006,0,0,0,0,3M India Ltd.,36277.0,Public Ltd.,1.020302e+10,1987.0,"Plastic furniture, floorings & miscellaneous i...",1.010120e+14,Manufacture of other plastics products n.e.c.,22209.0,2.020200e+10,Private (Foreign),Between 1986 and 1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaN,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE IN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1556.0,0,0,1556.0,1921.0,0,0,1921.0,1556.0,3289.0,(),"(('ASHOK LEYLAND LTD.', 'U'), ('3M INDIA LTD.'...","(('ASHOK LEYLAND LTD.', 'U'), ('3M INDIA LTD.'...",(),(),0,0,0,0,0,0,0,0,5,0,0,5,5,0,0,5,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,84.0,0.0,4.263014,0,1,0,0,2006,0,0,0,0,3M I

In [15]:
dir180CAR.to_csv("dir180CAR.csv")